# Exercise 2.03: Preparing the Quandl Data for Prediction

The goal of this exercise is to download an external dataset from the Quandl library and then prepare it so that it is ready for use in our linear regression models.

The following steps will help you to complete this exercise:

  > **Note**  
  > You need to install Qaundl library

1.- Download the data into a DataFrame using Quandl for the S&P 500 between 1950 and 2019. Its ticker is `“YALE/SPCOMP”`:

In [2]:
import quandl

dataFrame = quandl.get("YALE/SPCOMP", start_date="1950-01-01", end_date="2019-12-31")

2.- Use the `head()` method to visualize the columns inside the `data_frame.head()` DataFrame.

Output:

![Figure 2.18](img/fig2_18.jpg)

In [3]:
dataFrame.head()

,S&P Composite,Dividend,Earnings,CPI,Long Interest Rate,Real Price,Real Dividend,Real Earnings,Cyclically Adjusted PE Ratio
Year,,,,,,,,,
1950-01-31,16.88,1.15,2.33667,23.5,2.320000,189.693849,12.923455,26.259000,10.745733
1950-02-28,17.21,1.16,2.35333,23.5,2.340833,193.402318,13.035833,26.446222,10.911564
1950-03-31,17.35,1.17,2.37000,23.6,2.361667,194.149441,13.092498,26.520702,10.910947
1950-04-30,17.84,1.18,2.42667,23.6,2.382500,199.632624,13.204400,27.154849,11.178022
1950-05-31,18.44,1.19,2.48333,23.7,2.403333,205.476064,13.260115,27.671631,11.461543


A few features seem to highly correlate with each other. For instance, the `Real Dividend` column grows proportionally with `Real Price`. The ratio between them is not always similar, but they do correlate.

As regression is not about detecting the correlation between features, we would rather get rid of the features that we know are correlated and perform regression on the features that are non-correlated. In this case, we will keep the `Long Interest Rate`, `Real Price`, and `Real Dividend` columns.

3.- Keep only the relevant columns in the `Long Interest Rate`, `Real Price`, and `Real Dividend` DataFrames:

Output:

![Figure 2.19](img/fig2_19.jpg)

In [5]:
dataFrame = dataFrame[['Long Interest Rate', 'Real Price', 'Real Dividend']]
dataFrame

,Long Interest Rate,Real Price,Real Dividend
Year,,,
1950-01-31,2.320000,189.693849,12.923455
1950-02-28,2.340833,193.402318,13.035833
1950-03-31,2.361667,194.149441,13.092498
1950-04-30,2.382500,199.632624,13.204400
1950-05-31,2.403333,205.476064,13.260115
...,...,...,...
2019-08-31,1.630000,2982.540010,58.507324
2019-09-30,1.700000,3067.279487,58.853304
2019-10-31,1.710000,3055.689833,59.067968


You can see that the DataFrame contains a few missing values `NaN`. As regression doesn't work with missing values, we need to either replace them or delete them. In the real world, we will usually choose to replace them. In this case, we will replace the missing values by the preceding values using a method called **forward filling**.

4.- Replace the missing values with a forward filling.

Output:

![Figure 2.20](img/fig2_20.jpg)

In [6]:
dataFrame.fillna(method='ffill', inplace=True)
dataFrame

,Long Interest Rate,Real Price,Real Dividend
Year,,,
1950-01-31,2.320000,189.693849,12.923455
1950-02-28,2.340833,193.402318,13.035833
1950-03-31,2.361667,194.149441,13.092498
1950-04-30,2.382500,199.632624,13.204400
1950-05-31,2.403333,205.476064,13.260115
...,...,...,...
2019-08-31,1.630000,2982.540010,58.507324
2019-09-30,1.700000,3067.279487,58.853304
2019-10-31,1.710000,3055.689833,59.067968


Now that we have cleaned the missing data, we need to create our label. We want to predict the `Real Price` column 3 months in advance using the current `Real Price`, `Long Interest` Rate, and `Real Dividend` columns. In order to create our label, we need to shift the `Real Price` values up by three units and call it `Real Price Label`.

6.- Create the `Real Price Label` label by shifting `Real Price` by 3 months.

Output:

![Figure 2.21](img/fig2_21.jpg)

In [7]:
dataFrame['Real Price Label'] = dataFrame['Real Price'].shift(-3)
dataFrame

,Long Interest Rate,Real Price,Real Dividend,Real Price Label
Year,,,,
1950-01-31,2.320000,189.693849,12.923455,199.632624
1950-02-28,2.340833,193.402318,13.035833,205.476064
1950-03-31,2.361667,194.149441,13.092498,207.941560
1950-04-30,2.382500,199.632624,13.204400,190.450184
1950-05-31,2.403333,205.476064,13.260115,200.293903
...,...,...,...,...
2019-08-31,1.630000,2982.540010,58.507324,3187.956905
2019-09-30,1.700000,3067.279487,58.853304,3264.693814
2019-10-31,1.710000,3055.689833,59.067968,NaN


The side effect of shifting these values is that missing values will appear in the last three rows for `Real Price Label`, so we need to remove the last three rows of data. However, before that, we need to convert the features into a NumPy array and scale it. We can use the `drop` method of the DataFrame to remove the label column and the preprocessing function from `sklearn` to scale the features.

6.- Create a NumPy array for the features and scale it.

Output:

```
array([[-1.14834808, -1.13030753, -1.19145337],
       [-1.14109376, -1.12504171, -1.17960932],
       [-1.13383944, -1.12398084, -1.17363711],
       ...,
       [-1.36075461,  2.93924962,  3.67194058],
       [-1.32593387,  3.12706161,  3.7120735 ],
       [-1.3085235 ,  3.23602381,  3.75460521]])
```

In [9]:
import numpy as np
from sklearn import preprocessing

features = np.array(dataFrame.drop('Real Price Label', 1))
scaled = preprocessing.scale(features)
scaled

array([[-1.14834808, -1.13030753, -1.19145337],
       [-1.14109376, -1.12504171, -1.17960932],
       [-1.13383944, -1.12398084, -1.17363711],
       ...,
       [-1.36075461,  2.93924962,  3.67194058],
       [-1.32593387,  3.12706161,  3.7120735 ],
       [-1.3085235 ,  3.23602381,  3.75460521]])

The `1 in the second argument specifies that we are dropping columns. As the original DataFrame was not modified, the label can be directly extracted from it. Now that the features are scaled, we need to remove the last three values of the features as they are the features of the missing values in the label column. We will save them for later in the prediction part.

7.- Remove the last three values of the features array and save them into another array

Output:

```
array([[-1.14834808, -1.13030753, -1.19145337],
       [-1.14109376, -1.12504171, -1.17960932],
       [-1.13383944, -1.12398084, -1.17363711],
       ...,
       [-1.07522452,  2.86136628,  3.47166907],
       [-1.12049149,  2.78043332,  3.50007195],
       [-1.23888201,  2.83151407,  3.54006023]])
```

In [11]:
scaled_features_latest_3 = scaled[-3:]
scaled_features_latest_3 = scaled[:-3]
scaled_features_latest_3

array([[-1.14834808, -1.13030753, -1.19145337],
       [-1.14109376, -1.12504171, -1.17960932],
       [-1.13383944, -1.12398084, -1.17363711],
       ...,
       [-1.38861121,  2.97929738,  3.57121717],
       [-1.38861121,  2.83538088,  3.61285159],
       [-1.36423669,  2.95570629,  3.64931605]])

The `scaled_features` variable doesn't contain the three data points anymore as they are now in `scaled_features_latest_3`. Now we can remove the last three rows with missing data from the DataFrame, then convert the label into a NumPy array using `sklearn`.

8.- Remove the rows with missing data

Output:

![Figure 2.22](img/fig2_22.jpg)

In [12]:
dataFrame.dropna(inplace=True)
dataFrame

,Long Interest Rate,Real Price,Real Dividend,Real Price Label
Year,,,,
1950-01-31,2.320000,189.693849,12.923455,199.632624
1950-02-28,2.340833,193.402318,13.035833,205.476064
1950-03-31,2.361667,194.149441,13.092498,207.941560
1950-04-30,2.382500,199.632624,13.204400,190.450184
1950-05-31,2.403333,205.476064,13.260115,200.293903
...,...,...,...,...
2019-05-31,2.400000,2943.843050,57.437256,2982.540010
2019-06-30,2.060000,2979.816801,57.816670,3067.279487
2019-07-31,1.630000,3083.893573,58.112291,3055.689833


9.- Now let's see if we have accurately created our label.

Output:

![Figure 2.23](img/fig2_23.jpg)

In [14]:
label = np.array(dataFrame['Real Price Label'])
label

array([ 199.63262373,  205.47606414,  207.94155966,  190.45018423,
        200.29390288,  206.50815738,  213.31010407,  212.01882753,
        208.62952   ,  220.52387717,  226.06754864,  221.4040093 ,
        224.37244031,  223.60810193,  219.73345174,  223.60810193,
        233.39669189,  237.57801686,  235.46166718,  227.1757    ,
        233.29434264,  241.06749887,  238.48250951,  239.08499163,
        237.47913333,  237.3791    ,  242.96096   ,  248.06468315,
        249.05377678,  245.09740225,  239.95411536,  247.57013633,
        257.55998202,  259.91818947,  257.71002566,  258.03184662,
        245.32385263,  245.69085843,  236.00401493,  239.35438507,
        239.4463316 ,  228.4508461 ,  234.45145778,  240.52624535,
        243.76598662,  249.95094721,  255.44868996,  260.84825874,
        272.26684478,  282.05383792,  284.31183941,  295.7981948 ,
        301.68863346,  309.90923881,  317.1026806 ,  329.51875821,
        345.88604345,  352.11733333,  363.88754757,  361.01917

Our variable contains all the labels and is exactly the same as the `Real Price Label` column in the DataFrame.

Our next task is to separate the training and testing data from each other. As we saw in the Splitting Data into Training and Testing section, we will use $90\%$ of the data as the training data and the remaining $10\%$ as the test data.

10.- Split the `features` data into training and test sets using `sklearn`.

In [18]:
from sklearn import model_selection
(features_tr, features_ts, label_tr, label_ts) = model_selection.train_test_split(scaled, label, test_size=0.1, random_state=8)

ValueError: Found input variables with inconsistent numbers of samples: [840, 837]

By completing this exercise, we have learned all the required steps for data preparation before performing a regression.